In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


/Users/saadianz/Desktop/cny_calendar_scraper/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of CCNY Academic Calendar
url = "https://www.ccny.cuny.edu/registrar/academic-calendar"

# Fetch the page
response = requests.get(url)

# Check if request was successful
if response.status_code == 200:
    print("Page fetched successfully!")
    soup = BeautifulSoup(response.text, "html.parser")  # Parse the HTML
else:
    print("Failed to retrieve the page:", response.status_code)


Page fetched successfully!


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Fetch the CCNY Academic Calendar Webpage
url = "https://www.ccny.cuny.edu/registrar/academic-calendar"
response = requests.get(url)

# Step 2: Check if the page was successfully fetched
if response.status_code == 200:
    print("✅ Page fetched successfully!")
    soup = BeautifulSoup(response.text, "html.parser")
else:
    print(f"❌ Failed to fetch the page. Status code: {response.status_code}")
    exit()  # Stop execution if the page was not fetched

# Step 3: Find the Calendar Table inside the 'body-paragraph' class
calendar_div = soup.find("div", class_="body-paragraph")

if calendar_div:
    calendar_table = calendar_div.find("table")  # Find table inside the div
else:
    print("❌ No calendar section found on the page.")
    exit()

# Step 4: Extract Table Data
if calendar_table:
    rows = calendar_table.find_all("tr")  # Get all rows
    data = []

    for row in rows[1:]:  # Skip the header row
        cols = row.find_all("td")
        if len(cols) >= 2:
            date_text = cols[0].text.strip()  # First column = date
            description = cols[1].text.strip()  # Second column = event description
            data.append([date_text, description])

    # Step 5: Create a DataFrame
    df = pd.DataFrame(data, columns=["Date", "Text"])
    
    # Convert date column to datetime format
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # Add a "Day of the Week" column
    df["dow"] = df["Date"].dt.day_name()

    # Set Date as Index
    df.set_index("Date", inplace=True)

    # Show first few rows
    display(df.head())

    # Step 6: Save Data to CSV
    df.to_csv("ccny_calendar.csv")
    print("✅ Data saved to ccny_calendar.csv")
    
else:
    print("❌ No table found in the calendar section.")


✅ Page fetched successfully!
❌ No table found in the calendar section.


In [8]:
tables = soup.find_all("table")
print(f"✅ Found {len(tables)} tables on the page.")

# Print first few lines of each table
for i, table in enumerate(tables):
    print(f"\n🔹 Table {i+1}:\n")
    print(table.prettify()[:500])  # Show first 500 characters to inspect structure


✅ Found 0 tables on the page.


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Set Up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background (no GUI)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Automatically download and set up ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Step 2: Load the CCNY Academic Calendar Page
url = "https://www.ccny.cuny.edu/registrar/academic-calendar"
driver.get(url)

# Wait for JavaScript to load
driver.implicitly_wait(5)

# Step 3: Extract Page Source with JavaScript Content
soup = BeautifulSoup(driver.page_source, "html.parser")

# Step 4: Close the WebDriver
driver.quit()

# Step 5: Find and Extract the Calendar Data
calendar_div = soup.find("div", class_="body-paragraph")  # Adjust if needed

if calendar_div:
    calendar_table = calendar_div.find("table")
else:
    print("❌ No calendar section found on the page.")
    exit()

if calendar_table:
    rows = calendar_table.find_all("tr")
    data = []

    for row in rows[1:]:  # Skip header row
        cols = row.find_all("td")
        if len(cols) >= 2:
            date_text = cols[0].text.strip()  # First column = Date
            description = cols[1].text.strip()  # Second column = Event
            data.append([date_text, description])

    # Step 6: Create a DataFrame
    df = pd.DataFrame(data, columns=["Date", "Text"])

    # Convert Date to datetime format
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # Add "Day of the Week" column
    df["dow"] = df["Date"].dt.day_name()

    # Set Date as Index
    df.set_index("Date", inplace=True)

    # Show first few rows
    display(df.head())

    # Step 7: Save Data to CSV
    df.to_csv("ccny_calendar.csv")
    print("✅ Data saved to ccny_calendar.csv")
    
else:
    print("❌ No table found in the calendar section.")
